In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
from hftbacktest import NONE, NEW, HftBacktest, GTX, FeedLatency, BUY, SELL, Linear, Stat
from numba import njit
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV 

In [2]:
df = pd.concat([
              # pd.read_pickle('CollectData/data/btcusdt_20230228.pkl', compression='gzip'),
              pd.read_pickle('CollectData/data/btcusdt_20230227.pkl', compression='gzip')])

In [3]:
df = df[df.index < 1000]
# df

In [4]:
train, validate, test = np.split(df, [int(.6*len(df)), int(.8*len(df))])

In [5]:
# Support And Resistance


def leftLimitExtract(bids,leftLimit):
  res = []   
  for x in bids:
    # num1 = float(x)
    num1 = x
    if num1< leftLimit:
      break
    num2 = float(bids[x])
    # num2 = bids[x]
    res.append([num1,num2])
  return res


def rightLimitExtract(asks,rightLimit):
  # print(asks)
  res = []
  for x in asks:
    # num1 = float(x)
    num1 = x
    if num1 > rightLimit:
      break
    num2 = float(asks[x])
    # print(x)
    # num2 = asks[x]
    res.append([num1,num2])
  return res

def limitExtractDepth(bids,asks,limitPercent):

  currBidPrice = float(next(iter(bids)))
  leftLimit = currBidPrice - (currBidPrice*limitPercent)
  currAskPrice = float(next(iter(asks)))
  rightLimit = currAskPrice + (currAskPrice*limitPercent)
  bids = leftLimitExtract(bids,leftLimit)
  asks = rightLimitExtract(asks,rightLimit)
  return bids,asks



# @njit
# def histogram(data, weights, bins):
#     # Compute the range of the data
#     data_range = np.max(data) - np.min(data)
    
#     # Compute the width of each bin
#     bin_width = data_range / bins 

        
    
#     # Initialize the histogram counts
#     counts = np.zeros(bins, dtype=np.float64)
    
#     # Loop over the data points and accumulate the counts
#     for i in range(len(data)):
#         # Compute the bin index for this data point
#         bin_index = int((data[i] - np.min(data)) // bin_width)
        
#         # Add the weight of this data point to the appropriate bin
#         counts[bin_index] += weights[i]
    
#     # Compute the bin edges
#     bin_edges = np.linspace(np.min(data), np.max(data), bins+1)
    
#     return counts, bin_edges

def histogram(data, weights, bins):
    # Compute the range of the data
    data_range = np.max(data) - np.min(data)
    
    # Compute the width of each bin
    bin_width = data_range / bins 
    
    # Initialize the histogram counts
    counts = np.zeros(bins, dtype=np.float64)
    
    # Check if bin_width is 0
    if bin_width == 0:
        # Handle the case when bin_width is 0
        # For example, return an empty histogram
        return counts, np.array([np.min(data), np.max(data)])
    
    # Loop over the data points and accumulate the counts
    for i in range(len(data)):
        # Compute the bin index for this data point
        bin_index = int((data[i] - np.min(data)) // bin_width)
        
        # Add the weight of this data point to the appropriate bin
        counts[bin_index] += weights[i]
    
    # Compute the bin edges
    bin_edges = np.linspace(np.min(data), np.max(data), bins+1)
    
    return counts, bin_edges


def getSupportResistance(bids, asks, limitPercent):
    lineNumber = 5
    binNum=50   
    
    support = []    
    if bids:
        arr_bids = np.array(bids) 
        x_bids, y_bids = histogram(arr_bids[:, 0],arr_bids[:, 1], binNum)
        y_bids = [(y_bids[i] + y_bids[i-1])/2 for i in range(1, len(y_bids))]
        y_bids = np.asarray(y_bids, dtype=np.float64)
        x_bids = np.asarray(x_bids, dtype=np.float64)

        for i in range(y_bids.shape[0]):
            row = []
            row.append(y_bids[i])
            row.append(x_bids[i])
            support.append(row)
        support.sort(key=lambda x: x[1], reverse=True)
    
    resistance = []
    if asks:
        arr_asks = np.array(asks) 
        x_asks, y_asks = histogram(arr_asks[:, 0],arr_asks[:, 1], binNum)  
        y_asks = [(y_asks[i] + y_asks[i-1])/2 for i in range(1, len(y_asks))]
        y_asks = np.asarray(y_asks, dtype=np.float64)
        x_asks = np.asarray(x_asks, dtype=np.float64)
        for i in range(y_asks.shape[0]):
            row = []
            row.append(y_asks[i])
            row.append(x_asks[i])
            resistance.append(row)
        resistance.sort(key=lambda x: x[1], reverse=True)
    
    # print(resistance)
    return support[:lineNumber], resistance[:lineNumber]

In [6]:



def predict_njit(half_spread, lineNumber, skew, X, snapshot_df):
    snapshot_ = pd.read_pickle(snapshot_df, compression='gzip')
    hbt = HftBacktest(X,
                      tick_size=0.1,
                      lot_size=0.001,
                      maker_fee=-0.00005,
                      taker_fee=0.0007,
                      order_latency=FeedLatency(1),
                      asset_type=Linear,
                      snapshot=snapshot_)
    stat = Stat(hbt)
    
    while hbt.run:
        # Running interval in microseconds
        if not hbt.elapse(0.1 * 1e6):
            return False
        # Clear cancelled, filled or expired orders.
        hbt.clear_inactive_orders()
        
        binNum = 100
        
        
        ## Get market depth
        depth_market = 1001
        bid = []
        ask = []
        i = 0
        for tick_price in range(hbt.best_ask_tick, hbt.high_ask_tick + 1):
            if tick_price in hbt.ask_depth:
                ask.append([tick_price * hbt.tick_size, hbt.ask_depth[tick_price]])
                i += 1
                if i == depth_market:
                    break
        i = 0
        for tick_price in range(hbt.best_bid_tick, hbt.low_bid_tick - 1, -1):
            if tick_price in hbt.bid_depth:
                bid.append([tick_price * hbt.tick_size, hbt.bid_depth[tick_price]])
                i += 1
                if i == depth_market:
                    break

                    
        ## Define Parameter
        # max_position = 100000
        # order_interval = hbt.tick_size * 10
        # grid_num = 20
        # half_spread = hbt.tick_size * 20
        # depth = 0.05
        # lineNumber = 5
        # binNum=50 
        # skew = 1
        
                           
        
        
        ## Calculate alpha
        mid_price = (hbt.best_bid + hbt.best_ask) / 2.0 
#         buy = 0.0
#         sell = 0.0       
#         bid_arr = np.array(bid)
#         ask_arr = np.array(ask)
#         for i in range(bid_arr.shape[0]):
#             if bid_arr[i, 0] > mid_price * (1 - depth):
#                 buy += bid_arr[i, 1]

#         for i in range(ask_arr.shape[0]):
#             if ask_arr[i, 0] > mid_price * (1 + depth):
#                 sell += ask_arr[i, 1]
#         alpha = buy - sell
#         if alpha > 0:
#             skew = 1
#         else:
#             skew = -1

        reservation_price = mid_price + skew * hbt.tick_size * hbt.position 
        # print(reservation_price)
        # print(skew * hbt.position * hbt.tick_size)
        # return True
        
        
        
        limitPercent = (reservation_price + half_spread) / reservation_price - 1
        bid, ask = limitExtractDepth(bid,ask,limitPercent,reservation_price)
        supportLevel, resistanceLevel = getSupportResistance(bid, ask, limitPercent, lineNumber)   
        order_qty = 0.1 # np.round(notional_order_qty / mid_price / hbt.lot_size) * hbt.lot_size
        last_order_id = -1
        
        # Create a new grid for buy orders.
        new_bid_orders = Dict.empty(np.int64, np.float64)
        if hbt.position < max_position: # hbt.position * mid_price < max_notional_position
            p = hbt.position
            
            ##Support 
            
            for bid in supportLevel:                
                bid_order_price = bid[0] # supportLevel = [[0.01254, 11310.9], [0.0125, 3453.6]....]
                bid_order_tick = round(bid_order_price / hbt.tick_size)
                # Do not post buy orders above the best bid and below grid range.
                if bid_order_tick > hbt.best_bid_tick:
                    continue
                p += order_qty
                # Do not post buy orders that can exceed the maximum position.
                if p >= max_position:
                    continue
                # order price in tick is used as order id.
                new_bid_orders[bid_order_tick] = bid_order_price




        for order in hbt.orders.values():
            # Cancel if an order is not in the new grid.
            if order.side == BUY and order.cancellable and order.order_id not in new_bid_orders:
                hbt.cancel(order.order_id)
                last_order_id = order.order_id
        for order_id, order_price in new_bid_orders.items():
            # Post an order if it doesn't exist.
            if order_id not in hbt.orders:
                hbt.submit_buy_order(order_id, order_price, order_qty, GTX)
                last_order_id = order_id
                # print("test bid")
        
        # Create a new grid for sell orders.
        new_ask_orders = Dict.empty(np.int64, np.float64)
        if hbt.position > -max_position: # hbt.position * mid_price > -max_notional_position
            p = hbt.position
            
            ## RESISTANCE 
            for ask in resistanceLevel:  
                ask_order_price = ask[0] # resistanceLevel = [[0.01254, 11310.9], [0.0125, 3453.6]....]
                ask_order_tick = round(ask_order_price / hbt.tick_size)
                # Do not post sell orders below the best ask and above grid range
                if ask_order_tick < hbt.best_ask_tick:
                    continue
                p += order_qty
                # Do not post buy orders that can exceed the maximum position.
                if p <= -max_position:
                    continue
                # order price in tick is used as order id.
                new_ask_orders[ask_order_tick] = ask_order_price
                # print("check")
                
                



        for order in hbt.orders.values():
            # Cancel if an order is not in the new grid.
            if order.side == SELL and order.cancellable and order.order_id not in new_ask_orders:
                hbt.cancel(order.order_id)
                last_order_id = order.order_id
        for order_id, order_price in new_ask_orders.items():
            # Post an order if it doesn't exist.
            if order_id not in hbt.orders:
                hbt.submit_sell_order(order_id, order_price, order_qty, GTX)
                # print("check 2")
                last_order_id = order_id
                # print("test ask")
        
        # Elapse a process time
        if not hbt.elapse(.05 * 1e6):
            return False
        
        # All order requests are considered to be requested at the same time.
        # Wait until one of the order responses is received.
        if last_order_id >= 0:
            if not hbt.wait_order_response(last_order_id):
                return False
            
        stat.recorder.record(hbt)

    return stat.equity(None)

In [7]:
class Backtest:
    def __init__(self, snapshot_df=None, half_spread=None, lineNumber=None, skew=None):
        #order_interval, grid_num, half_spread, lineNumber, binNum, skew
        self.snapshot_df = snapshot_df
        self.half_spread = half_spread
        self.lineNumber = lineNumber
        self.skew = skew
        
    def set_params(self, half_spread, lineNumber, skew):
        self.half_spread = half_spread
        self.lineNumber = lineNumber
        self.skew = skew
        return self
        
    def get_params(self, deep=True):
        return { 'snapshot_df': self.snapshot_df, 
                'half_spread': self.half_spread, 
                'lineNumber': self.lineNumber, 
                'skew': self.skew }
        
    def fit(self, X, y=None):
        return self
    
    def predict(self, X):
        equity = predict_njit(self.half_spread, 
                              self.lineNumber, self.skew, X, self.snapshot_df )
        return equity
    
    def score(self, X):
        equity = self.predict(X)
        returns = (equity.diff() / 1000).fillna(0)

        return np.divide(returns.mean(), returns.std()) if (returns.std()) else 0

In [8]:
param_dist = {
    'half_spread': stats.uniform(1, 200),
    'lineNumber': stats.uniform(1, 10),
    'skew': stats.uniform(-5.0, 5)
}

search = RandomizedSearchCV(Backtest('CollectData/data/btcusdt_20230227.snapshot.pkl'),
                            cv=[(np.arange(len(train)), np.arange(len(validate)))],
                            param_distributions=param_dist,
                            verbose=1,
                            n_iter=1)

# param_grid = {
#     'half_spread': range(1, 201),
#     'lineNumber': range(1, 11),
#     'skew': range(-5, 6)
# }

# search = GridSearchCV(Backtest('CollectData/data/btcusdt_20230227.snapshot.pkl'),
#                       cv=[(np.arange(len(train)), np.arange(len(validate)))],
#                       param_grid=param_grid,
#                       verbose=1)

In [9]:
# import warnings

# # Ignore all warnings
# warnings.filterwarnings("ignore")
search.fit(train.values)

Fitting 1 folds for each of 1 candidates, totalling 1 fits


C:\Users\coppy\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\coppy\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py", line 765, in _score
    scores = scorer(estimator, X_test)
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\coppy\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_scorer.py", line 444, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\coppy\AppData\Local\Temp\ipykernel_15876\2354703380.py", line 30, in score
    equity = self.predict(X)
             ^^^^^^^^^^^^^^^
  File "C:\Users\coppy\AppData\Local\Temp\ipykernel_15876\2354703380.py", line 25, in predict
    equity = predict_njit(self.hal

RandomizedSearchCV(cv=[(array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 1...
                   estimator=<__main__.Backtest object at 0x00000174678E6390>,
                   n_iter=1,
                   param_distributions={'half_spread': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x00000174678E6350>,
                                        'lineNumber': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x00000174674530D0>,
                                        'skew': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x000001746795AAD0>},
                   verbose=1)

In [10]:
search.best_params_

{'half_spread': 3.035813512141238,
 'lineNumber': 1.9129822681697775,
 'skew': -4.685905767032752}

In [11]:
# with open('parameters.txt', 'w') as f:
#     f.write(str(search.best_params_))